In [2]:
#Run all this first
# Libraries
import requests
import pycurl
import os
import json
import re
import pandas as pd
from tqdm import tqdm
import sqlalchemy as db
#SQL Server [rename it as you need]
db_server = 'postgresql'
user = 'postgres'
password = 'admin'
ip = 'localhost'
db_name = 'fazenda'
# create the engine
engine = db.create_engine(f'{db_server}://{user}:{password}@{ip}/{db_name}')
#Type filter (will use it later)
types_df = ['int64', 'float64', "object", 'int64', "object", 'float64', "object", 'int64', 'int64', 'float64', "object", 'int64', 'int64', 'int64', 'int64', "object", "object", "object", "object", 'int64', "object", "object", "object", 'int64', "object", "object", "object", 'float64', 'int64', "object", 'float64', 'int64', "object", 'float64', "object", 'float64', 'int64', 'int64', "object", 'float64', "object", "object", "object",'int64', 'int64'] 

# Push (don't need to run, just if want to check)

## With Requests

In [13]:
#url = 'https://gatewayapi.prodam.sp.gov.br:443/financas/orcamento/sof/v3.0.1/empenhos?anoEmpenho=2019&mesEmpenho=10&numPagina=800'
#headers = {"Accept": "application/json", "Authorization": "Bearer xxxxxxxxxxx"}
#response = requests.get(url, headers = headers)

In [14]:
#response.json()

# Parallel

### If it's your first page, create it here

In [4]:
#Create your first page without Parallel
from multiprocess import Pool
from concurrent import futures
from concurrent.futures import ThreadPoolExecutor
    #from concurrent.futures import ProcessPoolExecutor
import threading
import time

#get_first_page
    # 1) Make your profile on https://api.prodam.sp.gov.br/store/;
    # 2) Generate the link you want on https://api.prodam.sp.gov.br/store/apis/info?name=SOF&version=v3.0.1&provider=admin;
    # 3) put your Authorization key below and edit the link to the one you got:
list1 = []
for i in tqdm(range(1)):
    for j in tqdm(range(1,13)):
        url = f'https://gatewayapi.prodam.sp.gov.br:443/financas/orcamento/sof/v3.0.1/empenhos?anoEmpenho=2018&mesEmpenho={j}&codOrgao=93&numPagina=' + str(i+1)
        headers = {"Accept": "application/json", "Authorization": "Bearer xxxxxxxxxxx"}
        response = requests.get(url, headers = headers)
        list1.append(response.json())

#Make Dataframe            
df_final = pd.DataFrame()
for k in range(len(list1)):
    df_temp = pd.DataFrame(list1[k]['lstEmpenhos'])
    for i in range(len(df_temp.columns)):
        df_temp[df_temp.columns[i]] = df_temp[df_temp.columns[i]].astype(types_df[i])
    df_final = df_final.append(df_temp)

#Add column Page
df_final['pagina'] = df_final.apply(lambda _: '', axis=1)
df_final['pagina'] = 1
df_final

#Send to SQL
conn = engine.connect()
df_final.to_sql('your_table_name_here', con = conn, index = False, if_exists = 'append')
df_sql = pd.read_sql('your_table_name_here', con = conn)[['pagina', 'mesEmpenho', 'anoEmpenho', 'codOrgao']]
conn.close()
df_sql

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.35s/it]


,pagina,mesEmpenho,anoEmpenho,codOrgao
0,1,1,2018,93
1,1,1,2018,93
2,1,1,2018,93
3,1,1,2018,93
4,1,1,2018,93
...,...,...,...,...
3595,1,12,2018,93
3596,1,12,2018,93
3597,1,12,2018,93
3598,1,12,2018,93


In [5]:
#Send to SQL
conn = engine.connect()
df_final.to_sql('your_table_name_here', con = conn, index = False, if_exists = 'append')
df_sql = pd.read_sql('your_table_name_here', con = conn)[['pagina', 'mesEmpenho', 'anoEmpenho', 'codOrgao']]
conn.close()
df_sql

,pagina,mesEmpenho,anoEmpenho,codOrgao
0,1,1,2018,93
1,1,1,2018,93
2,1,1,2018,93
3,1,1,2018,93
4,1,1,2018,93
...,...,...,...,...
4195,1,12,2018,93
4196,1,12,2018,93
4197,1,12,2018,93
4198,1,12,2018,93


In [ ]:
#Check DataFrame
#b = pd.DataFrame(list1[0]['lstEmpenhos'])
#b

In [ ]:
#Check Types 1
#for i in range(len(b.columns)):
#    b[b.columns[i]] = b[b.columns[i]].astype(types_df[i])
#x

In [ ]:
#Check Types 2
#mask = (pd.DataFrame(list1[0]['lstEmpenhos'], dtype = types_df) == pd.DataFrame(list1[1]['lstEmpenhos'])).sum() < 50
#pd.DataFrame(list1[0]['lstEmpenhos']).loc[:,mask].info()

In [ ]:
#Check types 3
#pd.DataFrame(list1[1]['lstEmpenhos']).dtypes.values

In [22]:
#Define function to Create your Table, just run it!
#def create_your_table(x):
#    conn = engine.connect()
#    df_final.to_sql(x, con = conn, index = False, if_exists = 'append')
#    conn.close()

In [24]:
#name your table and send it to DataBase
#create_your_table("your_table_name_here")

### Go for it!

In [6]:
#Import Libraries
from multiprocess import Pool
from concurrent import futures
from concurrent.futures import ThreadPoolExecutor
    #from concurrent.futures import ProcessPoolExecutor
import threading
import time

#get_data function
    #Change your URL and Headers, go to API STORE SOF to find what you need
def get_data(pair):
    try:
        i, j = pair
        url = f'https://gatewayapi.prodam.sp.gov.br:443/financas/orcamento/sof/v3.0.1/empenhos?anoEmpenho=2019&mesEmpenho={j}&codOrgao=93&numPagina={i+1}'
        headers = {"Accept": "application/json", "Authorization": "Bearer xxxxxxxxxxx"}
        response = requests.get(url, headers = headers)
        pages = pd.DataFrame(response.json()['lstEmpenhos'])
        if len(pages) > 0:
            pages['pagina'] = i
            return pages
        else:
            return response.json()
    except Exception as e: 
        return e

#Define pages
def num_pages(x):
    for i in (range(x)):
    #if want to select interval
#        if x < 370:
#            pass
        for j in (range(1,13)):
            parameters.append((i, j))

In [7]:
# Variables
list1 = []
parameters = []
pool = ThreadPoolExecutor(max_workers=6)
#Define pages
num_pages(2)

In [45]:
#if want to select parameters use this:
#parameters[::12]

In [9]:
# Connect to SQL to check duplicates
conn = engine.connect()
df_sql = pd.read_sql('your_table_name_here', con = conn)[['pagina', 'mesEmpenho', 'anoEmpenho', 'codOrgao']]
conn.close()
df_sql

,pagina,mesEmpenho,anoEmpenho,codOrgao
0,1,1,2018,93
1,1,1,2018,93
2,1,1,2018,93
3,1,1,2018,93
4,1,1,2018,93
...,...,...,...,...
4195,1,12,2018,93
4196,1,12,2018,93
4197,1,12,2018,93
4198,1,12,2018,93


In [10]:
#Now check your duplicates with SQL table
duplicates = list(df_sql.apply(lambda x : (int(x['pagina']), int(x['mesEmpenho'])), axis = 1).unique())
parameters2 = list(filter(lambda x : x not in duplicates, tqdm(parameters)))

100%|██████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<?, ?it/s]


In [11]:
#Run parallel and get your data!
list1 = list(tqdm(pool.map(get_data, tqdm(parameters2))))

100%|████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 1093.76it/s]
12it [00:04,  2.84it/s]


In [48]:
#If needed, test list1
#[i for i in list1 if isinstance(i,dict)]

In [12]:
#Create you final Dataframe
df_final = pd.DataFrame()
for df_temp in list1:
    try:
        for i in range(len(df_temp.columns)):
            df_temp[df_temp.columns[i]] = df_temp[df_temp.columns[i]].astype(types_df[i])
        df_final = df_final.append(df_temp)
    except:
        print(df_temp)
df_final

,numReserva,valPagoRestos,txtDescricaoFonteRecurso,codPrograma,codSubElemento,valPagoExercicio,codEmpresa,codCategoria,mesEmpenho,numContrato,...,anoContrato,anoEmpenho,codUnidade,txtDescricaoOrgao,valAnuladoEmpenho,txtGrupoDespesa,txtRazaoSocial,txtDescricaoItemDespesa,codGrupo,pagina
0,5033,0.0,Tesouro Municipal,3023,53,0.00,01,3,1,564.0,...,2019.0,2019,10,Fundo Municipal de Assistência Social,1310.86,Outras Despesas Correntes,UNIAO CIDADE LIDER PRO MELHORAMENTO DO BAIRRO,Outras Despesas - Lei de Diretrizes Orçamentárias,3,0
1,5069,0.0,Tesouro Municipal,3023,53,0.00,01,3,1,472.0,...,2019.0,2019,10,Fundo Municipal de Assistência Social,17389.40,Outras Despesas Correntes,CARITAS DIOCESANA DE CAMPO LIMPO-CDCL,Outras Despesas - Lei de Diretrizes Orçamentárias,3,0
2,5091,0.0,Tesouro Municipal,3023,53,0.00,01,3,1,558.0,...,2019.0,2019,10,Fundo Municipal de Assistência Social,0.00,Outras Despesas Correntes,ARRASTAO MOVIMENTO DE PROMOCAO HUMANA,Outras Despesas - Lei de Diretrizes Orçamentárias,3,0
3,5168,0.0,Tesouro Municipal,3023,53,0.00,01,3,1,476.0,...,2019.0,2019,10,Fundo Municipal de Assistência Social,2094.01,Outras Despesas Correntes,ASSOCIAÇÃO EDUCACIONAL E ASSISTENCIAL CASA DO ...,Outras Despesas - Lei de Diretrizes Orçamentárias,3,0
4,5193,0.0,Transferências Federais,3023,53,0.00,01,3,1,473.0,...,2019.0,2019,10,Fundo Municipal de Assistência Social,185614.36,Outras Despesas Correntes,ASSOCIAÇÃO DE MORADORES DO JARDIM COMERCIAL E ...,Outras Despesas - Lei de Diretrizes Orçamentárias,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,5698,0.0,Tesouro Municipal,3023,53,43127.48,01,3,12,4212.0,...,2014.0,2019,10,Fundo Municipal de Assistência Social,0.00,Outras Despesas Correntes,ACAO SOCIAL COMUNITARIA DO LAGEADO JOILSON DE ...,Outras Despesas - Lei de Diretrizes Orçamentárias,3,0
46,5700,0.0,Tesouro Municipal,3023,53,380265.59,01,3,12,30646.0,...,2015.0,2019,10,Fundo Municipal de Assistência Social,96882.29,Outras Despesas Correntes,ASSOCIAÇÃO DE MULHERES AMIGAS DE JOVA RURAL,Outras Despesas - Lei de Diretrizes Orçamentárias,3,0
47,5701,0.0,Tesouro Municipal,3023,53,615623.41,01,3,12,13524.0,...,2015.0,2019,10,Fundo Municipal de Assistência Social,143583.94,Outras Despesas Correntes,ACAO SOCIAL COMUNITARIA DO LAGEADO JOILSON DE ...,Outras Despesas - Lei de Diretrizes Orçamentárias,3,0
48,5704,0.0,Tesouro Municipal,3023,53,832673.62,01,3,12,13658.0,...,2015.0,2019,10,Fundo Municipal de Assistência Social,186461.42,Outras Despesas Correntes,ACAO SOCIAL COMUNITARIA DO LAGEADO JOILSON DE ...,Outras Despesas - Lei de Diretrizes Orçamentárias,3,0


In [10]:
# append to SQL
conn = engine.connect()
df_final.to_sql('your_table_name_here', con = conn, index = False, if_exists = 'append')
conn.close()

# Filter your table

In [ ]:
# Call DataFrame from SQL
#conn = engine.connect()
#df_final = pd.read_sql('as_social_2019', con = conn)


In [ ]:
#Filter
# Remember NOT to SUM all values; if you want final execution payment select only 12th months for each payment;
# CONCAT your years or do your magic;

In [ ]:
# Upload to SQL
#conn = engine.connect()
#df_final.to_sql('as_social_2019', con = conn, index = False, if_exists = 'append')
#conn.close()

# EXCEL_FILE

In [58]:
xl = pd.ExcelFile("C:/Users/user/Downloads/basedadosexecucaoConsolidados.xlsx")

In [59]:
xl.sheet_names
[u'_2003Em_Diante']
df = xl.parse("_2003Em_Diante")
df.head()
df.describe

<bound method NDFrame.describe of        DataInicial           DataFinal  Cd_AnoExecucao  Cd_Exercicio  \
0       2003-01-01 2003-12-31 23:59:59            2003          2003   
1       2003-01-01 2003-12-31 23:59:59            2003          2003   
2       2003-01-01 2003-12-31 23:59:59            2003          2003   
3       2003-01-01 2003-12-31 23:59:59            2003          2003   
4       2003-01-01 2003-12-31 23:59:59            2003          2003   
...            ...                 ...             ...           ...   
108521  2021-01-01 2021-04-30 23:59:59            2021          2021   
108522  2021-01-01 2021-04-30 23:59:59            2021          2021   
108523  2021-01-01 2021-04-30 23:59:59            2021          2021   
108524  2021-01-01 2021-04-30 23:59:59            2021          2021   
108525  2021-01-01 2021-04-30 23:59:59            2021          2021   

       Administracao  Cd_Orgao Sigla_Orgao  \
0             Direta        11    GAB-PREF   
1        

In [62]:
orgaos = [7,8,35,75,84,86, 87,89, 90, 91,93,96,99,'07','08']
df2= df[df.Cd_Orgao.isin(orgaos)]

In [63]:
df2.describe

<bound method NDFrame.describe of        DataInicial           DataFinal  Cd_AnoExecucao  Cd_Exercicio  \
8146    2003-01-01 2003-12-31 23:59:59            2003          2003   
8147    2003-01-01 2003-12-31 23:59:59            2003          2003   
8148    2003-01-01 2003-12-31 23:59:59            2003          2003   
8149    2003-01-01 2003-12-31 23:59:59            2003          2003   
8150    2003-01-01 2003-12-31 23:59:59            2003          2003   
...            ...                 ...             ...           ...   
108435  2021-01-01 2021-04-30 23:59:59            2021          2021   
108436  2021-01-01 2021-04-30 23:59:59            2021          2021   
108437  2021-01-01 2021-04-30 23:59:59            2021          2021   
108438  2021-01-01 2021-04-30 23:59:59            2021          2021   
108439  2021-01-01 2021-04-30 23:59:59            2021          2021   

       Administracao  Cd_Orgao Sigla_Orgao  \
8146          Direta        90      FUMCAD   
8147     

In [64]:
df2

,DataInicial,DataFinal,Cd_AnoExecucao,Cd_Exercicio,Administracao,Cd_Orgao,Sigla_Orgao,Ds_Orgao,Cd_Unidade,Ds_Unidade,...,Vl_Orcado_Atualizado,Vl_Congelado,Vl_Descongelado,Vl_CongeladoLiquido,Disponivel,Vl_ReservadoLiquido,Vl_EmpenhadoLiquido,Vl_Liquidado,Vl_Pago,DataExtracao
8146,2003-01-01,2003-12-31 23:59:59,2003,2003,Direta,90,FUMCAD,Fundo Municipal dos Direitos da Criança e do A...,10,Fundo Municipal dos Direitos da Criança e do A...,...,0.00,62307.45,62307.45,0.0,0.00,0.00,0.00,0.00,0.00,2011-03-30 19:31:49.250
8147,2003-01-01,2003-12-31 23:59:59,2003,2003,Direta,90,FUMCAD,Fundo Municipal dos Direitos da Criança e do A...,10,Fundo Municipal dos Direitos da Criança e do A...,...,22842.89,0.00,0.00,0.0,22842.89,22842.89,22842.89,0.00,0.00,2011-03-30 19:31:49.250
8148,2003-01-01,2003-12-31 23:59:59,2003,2003,Direta,90,FUMCAD,Fundo Municipal dos Direitos da Criança e do A...,10,Fundo Municipal dos Direitos da Criança e do A...,...,1720227.73,3273550.85,3273550.85,0.0,1720227.73,1720227.73,1720227.73,1550398.64,1242406.40,2011-03-30 19:31:49.250
8149,2003-01-01,2003-12-31 23:59:59,2003,2003,Direta,90,FUMCAD,Fundo Municipal dos Direitos da Criança e do A...,10,Fundo Municipal dos Direitos da Criança e do A...,...,40922.00,0.00,0.00,0.0,40922.00,40922.00,40922.00,40922.00,40922.00,2011-03-30 19:31:49.250
8150,2003-01-01,2003-12-31 23:59:59,2003,2003,Direta,90,FUMCAD,Fundo Municipal dos Direitos da Criança e do A...,10,Fundo Municipal dos Direitos da Criança e do A...,...,0.00,120000.00,120000.00,0.0,0.00,0.00,0.00,0.00,0.00,2011-03-30 19:31:49.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108435,2021-01-01,2021-04-30 23:59:59,2021,2021,Indireta,91,FMH,Fundo Municipal de Habitação,10,Fundo Municipal de Habitação,...,1000.00,1000.00,0.00,1000.0,0.00,0.00,0.00,0.00,0.00,2021-04-15 01:21:14.073
108436,2021-01-01,2021-04-30 23:59:59,2021,2021,Indireta,91,FMH,Fundo Municipal de Habitação,10,Fundo Municipal de Habitação,...,4000.00,4000.00,0.00,4000.0,0.00,0.00,0.00,0.00,0.00,2021-04-15 01:21:14.073
108437,2021-01-01,2021-04-30 23:59:59,2021,2021,Indireta,91,FMH,Fundo Municipal de Habitação,10,Fundo Municipal de Habitação,...,2029420.00,2029420.00,0.00,2029420.0,0.00,0.00,0.00,0.00,0.00,2021-04-15 01:21:14.073
108438,2021-01-01,2021-04-30 23:59:59,2021,2021,Indireta,91,FMH,Fundo Municipal de Habitação,10,Fundo Municipal de Habitação,...,1000000.00,1000000.00,0.00,1000000.0,0.00,0.00,0.00,0.00,0.00,2021-04-15 01:21:14.073
